In [1]:
!python --version

Python 3.12.6


# Pydantic: Simplimfying Data Validation in Python

Pydantic provides four ways to create schemas and perform validation and serialization:

* **BaseModel** — Pydantic's own super class with many common utilities available via instance methods.
* **Pydantic dataclasses** — a wrapper around standard dataclasses with additional validation performed.
* **TypeAdapter** — a general way to adapt any type for validation and serialization. This allows types like TypedDict and NamedTuple to be validated as well as simple types (like int or timedelta) — all types supported can be used with TypeAdapter.
* **validate_call** — a decorator to perform validation when calling a function.

In [9]:
import pydantic

print(pydantic.__version__)

2.9.2


#### 1. Example

In [ ]:
from datetime import date
from uuid import UUID, uuid4
from enum import Enum
from pydantic import BaseModel, EmailStr

class Department(Enum):
    HR = "HR"
    SALES = "SALES"
    IT = "IT"
    ENGINEERING = "ENGINEERING"

class Employee(BaseModel):
    employee_id: UUID = uuid4()
    name: str
    email: EmailStr
    date_of_birth: date
    salary: float
    department: Department
    elected_benefits: bool

* **employee_id**: This is the UUID for the employee you want to store information for. By using the UUID annotation, __Pydantic ensures this field is always a valid UUID__. Each instance of Employee will be assigned a UUID by default, as you specified by calling uuid4().
* **name**: The employee’s name, which Pydantic expects to be a string.
* **email**: __Pydantic will ensure that each employee email is valid by using Python’s email-validator library under the hood.__
* **date_of_birth**: Each employee’s date of birth must be a valid date, as annotated by date from Python’s datetime module. If you pass a string into date_of_birth, **Pydantic will attempt to parse and convert it to a date object**.
* **salary**: This is the employee’s salary, and it’s expected to be a float.
* **department**: Each employee’s department must be one of HR, SALES, IT, or ENGINEERING, as defined in your Department enum.
elected_benefits: This field stores whether the employee has elected benefits, and Pydantic expects it to be a Boolean.

Employee(
    name="Chris DeTuma",
    email="cdetuma@example.com",
    date_of_birth="1998-04-02",
    salary=123_000.00,
    department="IT",
    elected_benefits=True,
)